In [ ]:
import pandas as pd
df = pd.read_csv('../input/ballots/affinity.csv', index_col=0).fillna(0)
df.dropna(how='all', inplace=True)

In [ ]:
#Need a function to sum merge columns to avoid data fragmentation
def mergesum(df, parent, child):
    for row in range(df[parent].size):
        df[parent][row] = df[parent][row] + df[child][row]
    df.drop(child, axis=1, inplace=True)
    
def mergesums(df, parent, childs):
    for child in childs:
        try:
            mergesum(df, parent, child)
        except KeyError:
            continue

#Have to sum all names by hand lmfao
mergesums(df, 'citylion', ['city', 'citypedeleon', 'citylion1'])
mergesums(df, 'programmc', ['program', 'dchero', 'wrongaardvark', '\u2060programmc', "d'chero"])
mergesums(df, 'ahrimanne', ['ahri', 'ahrimanazu', 'pamplamanazu', ''])
mergesums(df, 'topher3001', ['topher', 'topha', 'toph'])
mergesums(df, 'sandfalls', ['swissandham', 'swiss', 'swiss&ham'])
mergesums(df, 'waffle4breakfast', ['waffle', 'waffles', 'pancake5lunch', 'crepe6dinner'])
mergesums(df, 'rakkwal', ['rakk', 'rakkwall'])
mergesums(df, 'kingcold64', ['kingcold', 'fladeedledoodle'])
mergesums(df, 'henrydraton', ['henry', 'henrydration'])
mergesums(df, 'godomasta', ['godo', '\u2060godomasta', 'godomasto', 'gododmasta'])
mergesums(df, 'specificlanguage', ['specific'])
mergesums(df, 'squareblob', ['square'])
df

In [ ]:
def predict(voters, candidates):
    votes = {}
    prediction = dict((candidate,0) for candidate in candidates)
    for voter in voters:
        votes[voter] = df.loc[voter, [candidate for candidate in candidates]].idxmax()
    print(votes)
    for vote in votes:
        prediction[votes[vote]] += 1
    return max(prediction, key=prediction.get)

voters = ['AllenY99', 'Fladeedledoddle', 'RavenMC_', 'Midnight_Jack', 'hayhayluke', 'MultiRobit2', 'smaldragon', 'ATJKPMC', 'Yellofishy', 'xkev320x',
         'meat312', 'Gjum', 'scramble0', 'Dr_Oracle', 'azkedar_', 'squareblob', 'morsden67', 'Ghunsan', 'MuffinPimp', 'wjkroeker',
         'SpaceVolcano', 'citylion1', 'Ham_Swiss', 'Neo355', 'ThePliv', 'Godomasta']
candidates = ['godomasta', 'squareblob']
predict(voters, candidates)

For my next trick, I'll try to convert the data to apply a ML model

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.svm import SVC

df1 = pd.read_csv('../input/ballots/participation.csv', index_col=0)
OHE = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
X = df1.iloc[:, 1:]
y = df1[['outcome']]
trainX, valX, trainy, valy = train_test_split(X, y)

def get_score(model):
    model.fit(trainX, trainy.values.ravel())
    predictions = model.predict(valX)
    score = f1_score(valy, predictions, average='macro')
    return f'MAE for {model} is:', score

def forecast(model, voters):
    X1 = pd.DataFrame([1 if name in voters else 0 for name in df1.iloc[:, 1:]], index=df1.columns[1:]).T
    model.fit(X, y)
    return model.predict(X1)

In [ ]:
forecast(XGBClassifier(n_estimators=800), ['AllenY99', 'Fladeedledoddle', 'RavenMC_', 'Midnight_Jack', 'hayhayluke', 'MultiRobit2', 'smaldragon', 'ATJKPMC', 'Yellofishy', 'xkev320x',
         'meat312', 'Gjum', 'scramble0', 'Dr_Oracle', 'azkedar_', 'squareblob', 'morsden67', 'Ghunsan', 'MuffinPimp', 'wjkroeker',
         'SpaceVolcano', 'Godomasta'])

In [ ]:
models = [RandomForestClassifier(), AdaBoostClassifier(), SVC(), XGBClassifier(n_estimators=800)]
for model in models:
    print(get_score(model))

In [ ]:
vote = [voter for voter in df1.columns if df1[voter][1] == 1]

In [ ]:
print(forecast(SVC(), vote))
print(predict(vote, ['icypenguin79', 'xkevio', 'crackerjack']))